# Run the other two notebooks first before running this!

This notebook assumes that you've 1) downloaded the initial dataset, and 2) ran the Branded Food Initial Data Cleaning notebook.

In [1]:
import os
import psycopg2
import pandas as pd
import psycopg2.extras as extras 
from config import config # This is a file to get params as a dict for the db login, make your own database.ini!
import numpy as np

In [2]:
reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv'))#.drop(columns='Unnamed: 0')
reduced_food.head()

C:\Users\alexa\AppData\Local\Temp\ipykernel_13432\3786388054.py:1: DtypeWarning: Columns (1,4,5,8,9,30) have mixed types. Specify dtype option on import or set low_memory=False.
  reduced_food = pd.read_csv(os.path.join('cleaned', 'branded_food_reduced.csv'))#.drop(columns='Unnamed: 0')


,fdc_id,gtin_upc,serving_size,serving_size_unit,household_serving_fulltext,package_weight,modified_date,available_date,preparation_state_code,short_description,...,insig_total_sugars,insig_calories_from_fat,insig_sugars,insig_fiber,insig_other_nutrients,household_serving_amount,household_serving_unit,category_id,brand_owner_id,brand_name_id
0,1105904,27000612323,15.0,ml,NaN,NaN,2020-10-02,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,164,0.0,NaN
1,1105905,51000198808,240.0,ml,NaN,NaN,2020-09-12,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,142,1.0,NaN
2,1105906,51000213273,440.0,g,NaN,NaN,2020-09-01,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,76,1.0,NaN
3,1105907,51000213303,440.0,g,NaN,NaN,2020-09-01,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,76,1.0,NaN
4,1105908,51000224637,240.0,ml,NaN,NaN,2020-10-03,2020-11-13,NaN,NaN,...,0,0,0,0,0,NaN,NaN,142,1.0,NaN


In [3]:
def connect():
    """Connects to the database and returns an sql cursor."""
    print('Attempting to connect to postgreSQL database...')
    connection = None
    crsr = None
    try:
        # Connect to db
        connection = psycopg2.connect(**config())
        print('Connected')
        # Get cursor and fetch version
        crsr = connection.cursor()
        crsr.execute('SELECT version()')
        db_version = crsr.fetchone() # Fetch first row?
        print('postgreSQL db version: {0}'.format(db_version))
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    return connection, crsr

def insert_values(conn, query, df, cols): 
    # Data to insert
    tuples = [tuple(x) for x in df[cols].fillna('NULL').to_numpy()] 

    
    # SQL query to execute  
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples)
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    print("the dataframe is inserted") 
    cursor.close() 


    

In [4]:
connection, cursor = connect()
connection, cursor

Attempting to connect to postgreSQL database...
Connected
postgreSQL db version: ('PostgreSQL 16.2, compiled by Visual C++ build 1937, 64-bit',)


(<connection object at 0x000001B3B44D7E10; dsn: 'user=postgres password=xxx dbname=FOOD_DATABASE host=localhost port=5432', closed: 0>,
 <cursor object at 0x000001B3C4997200; closed: 0>)

In [5]:
def run_command(conn, cmd, vars=None):
    try:
        with conn.cursor() as curs:
            curs.execute(cmd, vars)
    except(psycopg2.DatabaseError) as error:
        print(type(error), error)
        conn.rollback()
        
branded_food_schema = """
    CREATE TABLE branded_foods (
        fdc_id INTEGER PRIMARY KEY NOT NULL,
        gtin_upc VARCHAR(32),
        serving_size DECIMAL,
        serving_size_unit VARCHAR(4),
        package_weight VARCHAR(32), 
        available_date TIMESTAMP NOT NULL,
        insig_iron BOOLEAN,
        insig_calcium BOOLEAN,
        insig_cholesterol BOOLEAN,
        insig_dietary_fiber BOOLEAN,
        insig_trans_fat BOOLEAN, 
        insig_satured_fat BOOLEAN,
        insig_vitamin_d BOOLEAN,
        insig_potassium BOOLEAN, 
        insig_vitamin_a BOOLEAN,
        insig_vitamin_c BOOLEAN,
        insig_added_sugars BOOLEAN,
        insig_total_sugars BOOLEAN,
        insig_calories_from_fat BOOLEAN, 
        insig_sugars BOOLEAN, 
        insig_fiber BOOLEAN,
        category_id INTEGER,
        brand_owner_id INTEGER,
        brand_name_id INTEGER
    )
"""
run_command(connection, branded_food_schema)

<class 'psycopg2.errors.DuplicateTable'> relation "branded_foods" already exists



In [6]:
bool_cols = ['insig_iron', 
        'insig_calcium',
        'insig_cholesterol',
        'insig_dietary_fiber',
        'insig_trans_fat',
        'insig_satured_fat',
        'insig_vitamin_d', 
        'insig_potassium', 
        'insig_vitamin_a',
        'insig_vitamin_c',
        'insig_added_sugars',
        'insig_total_sugars',
        'insig_calories_from_fat',
        'insig_sugars', 
        'insig_fiber']

# Replace with TRUEs and FALSEs
pd.set_option('future.no_silent_downcasting', True) # Warning silence
reduced_food[bool_cols] = reduced_food[bool_cols].replace({0:'FALSE', 1:'TRUE'})

In [7]:
cols = ['fdc_id', 
        'gtin_upc', 
        'serving_size', 
        'serving_size_unit',
        'package_weight', 
        'available_date', 
        'insig_iron', 
        'insig_calcium',
        'insig_cholesterol',
        'insig_dietary_fiber',
        'insig_trans_fat',
        'insig_satured_fat',
        'insig_vitamin_d', 
        'insig_potassium', 
        'insig_vitamin_a',
        'insig_vitamin_c',
        'insig_added_sugars',
        'insig_total_sugars',
        'insig_calories_from_fat',
        'insig_sugars', 
        'insig_fiber',
        'category_id',
        'brand_owner_id',
        'brand_name_id']

In [8]:
query = "INSERT INTO %s(%s) VALUES (%s)" % ('branded_foods', ', '.join(cols), ', '.join(['%s'] * len(cols)))
query

'INSERT INTO branded_foods(fdc_id, gtin_upc, serving_size, serving_size_unit, package_weight, available_date, insig_iron, insig_calcium, insig_cholesterol, insig_dietary_fiber, insig_trans_fat, insig_satured_fat, insig_vitamin_d, insig_potassium, insig_vitamin_a, insig_vitamin_c, insig_added_sugars, insig_total_sugars, insig_calories_from_fat, insig_sugars, insig_fiber, category_id, brand_owner_id, brand_name_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

In [ ]:
# Gets row values for each value, None cast for nulls in SQL
rows = [tuple(x) for x in reduced_food[cols].replace([np.nan], [None]).head().to_numpy()] 

In [ ]:
for row in rows:
    run_command(connection, query, row) 